In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [2]:
from posarmctools.readdata import *
%load_ext autoreload
%autoreload 2

In [3]:
data_date = "2017_10_12_11_16_40"
data_dir = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-3/2017_10_12/" + data_date

In [4]:
blocksPerFile = 6
samplesPerRamp = 6000
rampsPerFile = 1500
samplesPerFile = samplesPerRamp * rampsPerFile

# selection in record
firstFile = 35
nbFiles = 14
lastFile = firstFile + nbFiles - 1
firstBlock = firstFile * blocksPerFile

### Read the data

In [5]:
timeSerie_A = np.zeros( samplesPerFile )
timeSerie_B = np.zeros( samplesPerFile )

adc_A = np.zeros( (nbFiles, samplesPerFile) )
adc_B = np.zeros( (nbFiles, samplesPerFile) )

In [6]:
lastBlock = (nbFiles-1) * blocksPerFile + firstBlock
for k in range( nbFiles ):
    
    # read the data
    nb = str( int( k * blocksPerFile + firstBlock) )
    filename = data_dir + "/record" + nb + ".bin"
    readFile( filename, samplesPerFile, timeSerie_A, timeSerie_B )
    
    print("k = {}, block = {} / {}".format(k, nb, lastBlock ))
    
    adc_A[ k, : ] = timeSerie_A
    adc_B[ k, : ] = timeSerie_B

k = 0, block = 210 / 288
k = 1, block = 216 / 288
k = 2, block = 222 / 288
k = 3, block = 228 / 288
k = 4, block = 234 / 288
k = 5, block = 240 / 288
k = 6, block = 246 / 288
k = 7, block = 252 / 288
k = 8, block = 258 / 288
k = 9, block = 264 / 288
k = 10, block = 270 / 288
k = 11, block = 276 / 288
k = 12, block = 282 / 288
k = 13, block = 288 / 288


In [7]:
A_reshaped = adc_A.reshape(nbFiles * rampsPerFile, samplesPerRamp)
B_reshaped = adc_B.reshape(nbFiles * rampsPerFile, samplesPerRamp)

**Check that the first ramp is an up ramp**

In [8]:
plt.figure()
plt.plot( A_reshaped[0, :] )
plt.title("first acquisition\n" + data_date)
plt.grid()

**Set the shifted flag properly**

In [ ]:
shifted = 1

In [ ]:
coupling_A = np.average(A_reshaped, 0)
#A_reshaped -= coupling_A

In [ ]:
plt.figure()
plt.plot( coupling_A )
plt.title("coupling\n" + data_date)
plt.grid()

In [ ]:
ifft_a = np.fft.ifft( (A_reshaped[::,0:3000]), axis = 1 )

In [ ]:
plt.figure()
vmin = -10
vmax = 10
nbPointsInRange = 500
plt.plot(  20 * np.log10( np.abs( ifft_a[2500,0:nbPointsInRange] ) ) )
plt.plot( np.ones( nbPointsInRange ) * vmin, "b" )
plt.plot( np.ones( nbPointsInRange ) * vmax, "r" )

In [ ]:
plt.matshow( 20 * np.log10( np.abs( ifft_a[:,100:500] ) ), 
            aspect='auto', vmin=-20, vmax=20, cmap='jet' )
plt.title("Track 1 - record 2017_10_12_11_16_40\nFiles {} to {}".format(firstFile, lastFile) )

In [ ]:
plt.figure()
azStart = 10000
azStop = 11000
rangeCell = 192
plt.plot( np.angle( ifft_a[azStart:azStop,rangeCell] ) )
plt.title("ifft angle, az = [ {}, {} ], range cell = {}\n".format( azStart, azStop, rangeCell ) + data_date)

In [ ]:
np.save( data_dir + "/selection_files_{}_{}".format(firstFile, lastFile), A_reshaped )
np.save( data_dir + "/selection_files_{}_{}_shifted".format(firstFile, lastFile), shifted )

## Compute RD

In [ ]:
import sys

In [ ]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *
from posarutils.other.PosarMCParameters import *

In [ ]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarMCParameters( params_filename )

In [ ]:
RDc = build_rd_from_data( params, A_reshaped, shifted )

In [ ]:
RDc_0 = build_rd_from_data( params, A_reshaped, 0 )

In [ ]:
coupling_RD_local = np.average(RDc, 0)
np.save( data_dir + "/coupling_RD_files_{}_{}".format(firstFile, lastFile), coupling_RD_local )

In [ ]:
plt.figure()
plt.plot(np.abs(coupling_RD_local))
plt.grid()

In [ ]:
np.save(data_dir + '/RD_files_{}_{}'.format(firstFile, lastFile), RDc)

In [ ]:
ifft_RD = np.fft.ifft(RDc, axis = 1)

In [ ]:
nbPointsInRange = 300
x0 = 0
x1 = nbPointsInRange
y0 = 0
y1 = 0
extent = [ x0, x1, y0, y1 ]
rangeStart = 135
rangeStop = 250
azStart = 6000
azStop = 15000

In [ ]:
plt.figure()
plt.matshow( 20 * np.log10( np.abs( ifft_RD[azStart:azStop, rangeStart:rangeStop] ) ),
            aspect='auto')
plt.title(data_date)
ax = plt.gca()
#ax.axes.set_xlim(130,170)
#ax.axes.set_ylim(280,376)
plt.colorbar()

In [ ]:
ifft_RD_0 = np.fft.ifft(RDc_0, axis = 1)

In [ ]:
plt.figure()
plt.matshow( 20 * np.log10( np.abs( ifft_RD_0[azStart:azStop, rangeStart:rangeStop] ) ),
            aspect='auto')
plt.title(data_date)
ax = plt.gca()
#ax.axes.set_xlim(130,170)
#ax.axes.set_ylim(280,376)
plt.colorbar()

In [ ]:
rangeToPlot = rangeStart + 57
azStart = 9500
azStop = 11500

plt.figure()
plt.suptitle( "angle / real / imag (range, azimut) up ramp and down ramp\n" + data_date )

plt.subplot(311)
plt.plot( np.angle( ifft_RD[azStart:azStop, rangeToPlot] ), label='up ramp' )
plt.plot( np.angle( ifft_RD_0[azStart:azStop, rangeToPlot] ), label='down ramp' )
plt.legend()
plt.grid()

plt.subplot(312)
plt.plot( np.real( ifft_RD[azStart:azStop, rangeToPlot] ), label='up_ramp' )
plt.plot( np.real( ifft_RD_0[azStart:azStop, rangeToPlot] ), label='down ramp' )
plt.legend()
plt.grid()

plt.subplot(313)
plt.plot( np.imag( ifft_RD[azStart:azStop, rangeToPlot] ), label='up ramp' )
plt.plot( -np.imag( ifft_RD_0[azStart:azStop, rangeToPlot] ), label='down ramp' )
plt.legend()
plt.grid()

In [ ]:
plt.figure()
plt.suptitle( "unwrapped angle (range, azimut) up ramp and down ramp\n" + data_date )

plt.subplot(211)
plt.plot( np.unwrap( np.angle( ifft_RD[azStart:azStop, rangeToPlot] ) ), label='up ramp' )
plt.plot( np.unwrap( np.angle( ifft_RD_0[azStart:azStop, rangeToPlot] ) ), label='down ramp' )
plt.legend()
plt.grid()

plt.subplot(212)
plt.plot( np.unwrap( np.angle( ifft_RD[azStart:azStop, rangeToPlot] ) ), label='up ramp' )
plt.plot( np.unwrap( np.angle( np.conjugate( ifft_RD_0[azStart:azStop, rangeToPlot] ) ) ), label='down ramp (conjugate)' )
plt.legend()
plt.grid()

In [ ]:
RD_upNdown = build_rd_from_data_upNDown( params, A_reshaped, shifted=1 )

In [ ]:
ifft_RD_upNdown = np.fft.ifft(RD_upNdown, axis = 1)

In [ ]:
plt.figure()
plt.matshow( 20 * np.log10( np.abs( ifft_RD[azStart:azStop, rangeStart:rangeStop] ) ),
            aspect='auto')
plt.title(data_date)
ax = plt.gca()
#ax.axes.set_xlim(130,170)
#ax.axes.set_ylim(280,376)
plt.colorbar()

In [ ]:
np.save(data_dir + '/RD_files_{}_{}_upNdown'.format(firstFile, lastFile), RD_upNdown)

In [ ]:
coupling_RD_upNdown = np.average(RD_upNdown, 0)
np.save( data_dir + "/coupling_RD_files_{}_{}_upNdown".format(firstFile, lastFile), coupling_RD_upNdown )